In [3]:
import pandas as pd 
from datetime import datetime


df= pd.read_csv('dataELO.csv')
df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')
df.tail(20)

C:\Users\Cristina Jorge\AppData\Local\Temp\ipykernel_6828\2335523116.py:6: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')


,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J
22572,Série B,2023.0,2023-08-12,Sampaio Corrêa,1.0,1.0,CRB,E,1,1,222.0
22573,Série C,2023.0,2023-08-13,Paysandu,4.0,2.0,Náutico,VC,3,0,169.0
22574,Série A,2023.0,2023-08-13,Flamengo,1.0,1.0,São Paulo,E,1,1,181.0
22575,Série C,2023.0,2023-08-13,São José-RS,3.0,3.0,Figueirense,E,1,1,168.0
22576,Série C,2023.0,2023-08-13,Manaus,1.0,1.0,Remo,E,1,1,167.0
22577,Série C,2023.0,2023-08-13,América-RN,2.0,0.0,Pouso Alegre,VC,3,0,166.0
22578,Série C,2023.0,2023-08-13,CSA,1.0,1.0,Botafogo-PB,E,1,1,165.0
22579,Série B,2023.0,2023-08-13,Vitória,1.0,0.0,Ceará,VC,3,0,225.0
22580,Série B,2023.0,2023-08-13,Chapecoense,0.0,1.0,Atlético-GO,VV,0,3,226.0
22581,Série B,2023.0,2023-08-13,Guarani,0.0,1.0,Juventude,VV,0,3,227.0


In [4]:
#This function updates the Elo ratings and creates a ranking

timestamp_df = pd.DataFrame(columns=['Timestamp'])
def update_elo(df):
    #Gives different weights to each competition
    k_factor_map = {
                    'Série C': 10, 
                    'Série B': 20, 
                    'CBR': 25, 
                    'Série A': 30, 
                    }
    teams = {}
    elo_ranking = {}
    average_elo = {}
    team_games = {}
    team_victories = {}
    team_losses = {}
    team_goals = {}
    team_draws = {}
    last_competition = {}
    current_year = datetime.now().year
    
    #Creates a list of teams and gives then an initial rating of 1500
    for index, row in df.iterrows():
        home_team, away_team = row['Casa'], row['Fora']
        comp_level = row['Comp']
        match_date = row['Data'].year
        k_factor = k_factor_map.get(comp_level, 50) 
        if home_team not in teams:
            teams[home_team] = 1500
            elo_ranking[home_team] = teams[home_team]
        if away_team not in teams:
            teams[away_team] = 1500
            elo_ranking[away_team] = teams[away_team]
        home_elo, away_elo = teams[home_team], teams[away_team]
        home_score, away_score = row['GC'], row['GF']
        result = row['Res']
        # Only update the last competition if it's not 'CBR' and the match is in the current year
        if comp_level != 'CBR' and match_date == current_year:
            last_competition[home_team] = comp_level
            last_competition[away_team] = comp_level

        #Gives different weights for home and away games
        if result == 'VC':
            home_result, away_result = 1, 0
        elif result == 'VV':
            home_result, away_result = 0, 1
        else:
            home_result, away_result = 0.5, 0.5
        home_expected = 1 / (1 + 10 ** ((away_elo - home_elo - 100) / 400))
        away_expected = 1 / (1 + 10 ** ((home_elo - away_elo + 100) / 400))
        
        #Calculates the number of goals
        if home_team in team_goals:
            team_goals[home_team] += home_score
        else:
            team_goals[home_team] = home_score

        if away_team in team_goals:
            team_goals[away_team] += away_score
        else:
            team_goals[away_team] = away_score
       # Update the number of victories, losses, and draws for each team
        if result == 'VC':
            home_result, away_result = 1, 0
            if home_team in team_victories:
                team_victories[home_team] += 1
            else:
                team_victories[home_team] = 1
            if away_team in team_losses:
                team_losses[away_team] += 1
            else:
                team_losses[away_team] = 1
        elif result == 'VV':
            home_result, away_result = 0, 1
            if home_team in team_losses:
                team_losses[home_team] += 1
            else:
                team_losses[home_team] = 1
            if away_team in team_victories:
                team_victories[away_team] += 1
            else:
                team_victories[away_team] = 1
        else:
            home_result, away_result = 0.5, 0.5
            if home_team in team_draws:
                team_draws[home_team] += 1
            else:
                team_draws[home_team] = 1
            if away_team in team_draws:
                team_draws[away_team] += 1
            else:
                team_draws[away_team] = 1
        # Update the number of games played for each team
        if home_team in team_games:
            team_games[home_team] += 1
        else:
            team_games[home_team] = 1
        if away_team in team_games:
            team_games[away_team] += 1
        else:
            team_games[away_team] = 1
        
        # Calculate the goal difference and adjust the K factor accordingly
        goal_diff = abs(home_score - away_score)
        if goal_diff >= 4:
            k_factor = k_factor + 0.75 + (goal_diff - 4) / 8
        elif goal_diff == 3:
            k_factor = k_factor + 0.75
        elif goal_diff == 2:
            k_factor = k_factor + 0.5
        else:
            k_factor = k_factor
            
        # Calculate the rating difference factor
        rating_diff = abs(home_elo - away_elo)
        if rating_diff > 400:
            rating_diff = 400
        rating_factor = rating_diff / 400 + 1
            
        teams[home_team] = home_elo + rating_factor * k_factor * (home_result - home_expected)
        teams[away_team] = away_elo + rating_factor * k_factor * (away_result - away_expected)
        teams[home_team] = round(teams[home_team], 2)
        teams[away_team] = round(teams[away_team], 2)
        elo_ranking[home_team] = teams[home_team]
        elo_ranking[away_team] = teams[away_team]
        df.at[index, 'Home_Elo'] = home_elo
        df.at[index, 'Away_Elo'] = away_elo
        df.at[index, 'Home_New_Elo'] = teams[home_team]
        df.at[index, 'Away_New_Elo'] = teams[away_team]

        
        for team, elo in teams.items():
            if team in average_elo:
                average_elo[team].append(elo)
            else:
                average_elo[team] = [elo]

    # Compute average Elo for each team
    for team, elo_list in average_elo.items():
        average_elo[team] = round(sum(elo_list) / len(elo_list), 2)

    #Createa the Datasets
    elo_df = pd.DataFrame.from_dict(elo_ranking, orient='index', columns=['Elo'])
    elo_df = elo_df.sort_values(by='Elo', ascending=False)
    elo_df.index.name = 'Time'
    elo_df.reset_index(inplace=True)
    #elo_df = elo_df[~elo_df['Time'].str.contains(r'\(ARG\)|\(BOL\)|\(CHI\)|\(COL\)|\(EQU\)|\(MEX\)|\(PAR\)|\(PER\)|\(URU\)|\(VEN\)', regex=True)]
    elo_df['Jogos'] = elo_df['Time'].map(team_games) 
    elo_df = elo_df[elo_df['Jogos'] >= 30]
    elo_df['Rank'] = elo_df['Elo'].rank(ascending=False, method='dense')
    elo_df['Média'] = elo_df['Time'].map(average_elo).round(2)
    elo_df['#'] = elo_df['Média'].rank(ascending=False, method='dense')
    elo_df['Gols'] = elo_df['Time'].map(team_goals)
    elo_df['Vitórias'] = elo_df['Time'].map(team_victories)
    elo_df['Derrotas'] = elo_df['Time'].map(team_losses)
    elo_df['Empates'] = elo_df['Time'].map(team_draws)
    elo_df['Série'] = elo_df.apply(lambda row: last_competition.get(row['Time'], '-'), axis=1)    
    timestamp_now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    timestamp_df.loc[len(timestamp_df)] = [timestamp_now]

    
    return df, elo_df

df, elo_df = update_elo(df)
df.to_csv('dataELO.csv', index=False)
elo_df.to_json('dataELOranking.json', orient='records')
timestamp_df.to_json('timestamps.json')



In [10]:
filt = (elo_df['Série'] == "-")
elo_df[filt]

,Time,Elo,Jogos,Rank,Média,#,Gols,Vitórias,Derrotas,Empates,Série
51,Nacional-AM,1532.19,58,44.0,1525.36,35.0,83.0,19,18,21,-
59,Santa Cruz,1521.30,526,48.0,1514.56,45.0,691.0,186,198,142,-
60,Novo Hamburgo,1518.73,37,49.0,1508.03,59.0,48.0,19,13,5,-
61,Juazeirense,1516.91,34,50.0,1490.70,86.0,33.0,8,14,12,-
62,Americano,1516.76,66,51.0,1523.02,36.0,94.0,30,24,12,-
...,...,...,...,...,...,...,...,...,...,...,...
411,Campinense,1402.62,150,128.0,1456.12,114.0,171.0,46,68,36,-
412,Fast Clube,1400.57,47,129.0,1430.75,129.0,56.0,14,23,10,-
414,Mogi Mirim,1389.92,173,130.0,1425.75,131.0,184.0,47,76,50,-
416,Guaratinguetá,1380.26,240,131.0,1446.20,124.0,290.0,80,105,55,-


In [11]:
elo_df.to_json('dataELOranking.json', orient='records')


In [12]:
timestamp_df

,Timestamp
0,2023-08-16 14:50:52


In [30]:
filt = ((df['Casa'] == 'Corinthians') | (df['Fora'] == 'Corinthians'))
df[filt].tail(30)

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J,Home_Elo,Away_Elo,Home_New_Elo,Away_New_Elo
21824,Série A,2022.0,2022-10-26,Corinthians,0,2,Fluminense,VV,0,3,333.0,1851.37,1811.72,1828.21,1834.88
21839,Série A,2022.0,2022-10-29,Goiás,0,0,Corinthians,E,1,1,341.0,1744.52,1828.21,1743.67,1829.06
21850,Série A,2022.0,2022-11-02,Flamengo,1,2,Corinthians,VV,0,3,346.0,1920.86,1829.06,1893.16,1856.76
21857,Série A,2022.0,2022-11-05,Corinthians,1,0,Ceará,VC,3,0,354.0,1856.76,1668.28,1863.81,1661.23
21877,Série A,2022.0,2022-11-09,Coritiba,2,2,Corinthians,E,1,1,369.0,1697.30,1863.81,1701.32,1859.79
21885,Série A,2022.0,2022-11-13,Corinthians,0,1,Atlético-MG,VV,0,3,375.0,1859.79,1801.41,1835.27,1825.93
21984,CBR,2023.0,2023-04-12,Remo,2,0,Corinthians,VC,3,0,68.0,1542.05,1835.27,1575.31,1802.01
22011,Série A,2023.0,2023-04-16,Corinthians,2,1,Cruzeiro,VC,3,0,9.0,1802.01,1668.42,1810.28,1660.15
22030,Série A,2023.0,2023-04-23,Goiás,3,1,Corinthians,VC,3,0,18.0,1687.70,1810.28,1708.92,1789.06
22041,CBR,2023.0,2023-04-26,Corinthians,2,0,Remo,VC,3,0,85.0,1789.06,1575.31,1794.58,1569.79


In [228]:
unique_values = df['Fora','Casa'].unique()
unique_values_sorted = sorted(unique_values)
print(unique_values_sorted)

['ABC', 'ASA', 'Alianza Lima (PER)', 'Altos', 'Always Ready (BOL)', 'America-RJ', 'Americano-RJ', 'América (COL)', 'América (MEX)', 'América (Teófilo Otoni)', 'América-MG', 'América-RN', 'América-SP', 'Anapolina', 'Aparecidense', 'Araxá', 'Argentinos Jrs (ARG)', 'Arsenal (ARG)', 'Athletic', 'Athletico-PR', 'Atlas', 'Atlético Acreano', 'Atlético Cearense', 'Atlético Nacional (COL)', 'Atlético PR', 'Atlético Sorocaba', 'Atlético Tucumán (COL)', 'Atlético-GO', 'Atlético-MG', 'Aucas (EQU)', 'Audax-RJ', 'Avaí', 'Bahia', 'Bangu', 'Baraúnas', 'Barcelona (EQU)', 'Barra Mansa', 'Barras', 'Barueri', 'Betim', 'Binacional', 'Boa', 'Boavista', 'Boca Juniors (ARG)', 'Bolívar (BOL)', 'Bonsucesso', 'Botafogo', 'Botafogo-PB', 'Botafogo-SP', 'Bragantino', 'Brasil de Pelotas', 'Brasil-RS', 'Brasiliense', 'Brusque', 'CRAC', 'CRB', 'CSA', 'Cabofriense', 'Caldense', 'Campinense', 'Capivariano', 'Caracas (VEN)', 'Cardoso Moreira', 'Catanduvense', 'Caxias', 'Ceará', 'Ceilândia', 'Central (ARG)', 'Cerro Porteñ

In [230]:
df['Temp'] = df['Temp'].astype(str)
df

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J,Home_Elo,Away_Elo,Home_New_Elo,Away_New_Elo
0,Paulista,2003.0,2003-01-25,Marília,0,2,Corinthians,VV,0,3,2,1500.0,1500.0,1487.0,1513.0
1,Paulista,2003.0,2003-01-26,União São João,0,2,América-SP,VV,0,3,9,1500.0,1500.0,1487.0,1513.0
2,Paulista,2003.0,2003-01-26,Portuguesa,0,3,São Caetano,VV,0,3,8,1500.0,1500.0,1487.0,1513.0
3,Paulista,2003.0,2003-01-26,Paulista,2,1,São Paulo,VC,3,0,7,1500.0,1500.0,1507.0,1493.0
4,Paulista,2003.0,2003-01-26,Guarani,4,0,União Barbarense,VC,3,0,3,1500.0,1500.0,1507.0,1493.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16694,Série A,2023.0,2023-06-25,Palmeiras,0,1,Botafogo,VV,0,3,116,1809.0,1768.0,1778.0,1799.0
16695,Série A,2023.0,2023-06-25,Red Bull Bragantino,2,0,Goiás,VC,3,0,118,1683.0,1596.0,1696.0,1583.0
16696,Série A,2023.0,2023-06-25,Grêmio,5,1,Coritiba,VC,3,0,119,1759.0,1480.0,1766.0,1473.0
16697,Série A,2023.0,2023-06-25,Santos,2,3,Flamengo,VV,0,3,117,1621.0,1712.0,1596.0,1737.0


In [2]:
filt = (df['Casa'] == 'Corinthians') | (df['Fora'] == 'Corinthians')
df.loc[filt].tail(31)

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J
17107,CBR,2022,2022-10-19,Flamengo,1,1,Corinthians,E,1,1,121
17119,Série A,2022,2022-10-22,Santos,0,1,Corinthians,VV,0,3,324
17134,Série A,2022,2022-10-26,Corinthians,0,2,Fluminense,VV,0,3,333
17147,Série A,2022,2022-10-29,Goiás,0,0,Corinthians,E,1,1,341
17153,Série A,2022,2022-11-02,Flamengo,1,2,Corinthians,VV,0,3,346
17165,Série A,2022,2022-11-05,Corinthians,1,0,Ceará,VC,3,0,354
17184,Série A,2022,2022-11-09,Coritiba,2,2,Corinthians,E,1,1,369
17192,Série A,2022,2022-11-13,Corinthians,0,1,Atlético-MG,VV,0,3,375
17263,CBR,2023,2023-04-12,Remo,2,0,Corinthians,VC,3,0,68
17291,Série A,2023,2023-04-16,Corinthians,2,1,Cruzeiro,VC,3,0,9


In [35]:
filt = ((df['Casa'] == 'Corinthians') | (df['Fora'] == 'Corinthians'))
df.loc[filt].tail(30)

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J,Home_Elo,Away_Elo,Home_New_Elo,Away_New_Elo
17074,Série A,2022,2022-10-08,Corinthians,2,1,Athletico-PR,VC,3,0,301,1798.90,1774.67,1812.83,1760.74
17087,CBR,2022,2022-10-12,Corinthians,0,0,Flamengo,E,1,1,120,1812.83,1879.64,1811.16,1881.31
17107,CBR,2022,2022-10-19,Flamengo,1,1,Corinthians,E,1,1,121,1893.26,1811.16,1884.57,1819.85
17119,Série A,2022,2022-10-22,Santos,0,1,Corinthians,VV,0,3,324,1743.20,1819.85,1717.77,1845.28
17132,Série A,2022,2022-10-26,Corinthians,0,2,Fluminense,VV,0,3,333,1845.28,1785.18,1811.95,1818.51
17146,Série A,2022,2022-10-29,Goiás,0,0,Corinthians,E,1,1,341,1734.19,1811.95,1732.66,1813.48
17152,Série A,2022,2022-11-02,Flamengo,1,2,Corinthians,VV,0,3,346,1918.25,1813.48,1879.65,1852.08
17161,Série A,2022,2022-11-05,Corinthians,1,0,Ceará,VC,3,0,354,1852.08,1623.62,1860.32,1615.38
17184,Série A,2022,2022-11-09,Coritiba,2,2,Corinthians,E,1,1,369,1697.71,1860.32,1702.73,1855.30
17192,Série A,2022,2022-11-13,Corinthians,0,1,Atlético-MG,VV,0,3,375,1855.30,1772.11,1819.46,1807.95


In [ ]:
7